In [1]:
# ==============================================
# 🚚 Truck Utilization & Visibility Simulator
# ==============================================

from IPython.display import display, HTML, FileLink, clear_output
import pandas as pd
import io
import ipywidgets as widgets
import matplotlib.pyplot as plt

# --- UI Header ---
display(HTML("""
<h1>🚚 Truck Utilization & Visibility Simulator</h1>
<p>This interactive tool estimates truck requirements and utilization based on order data.<br>
Upload your Excel file or download the template to begin.</p>
"""))

# --- File upload and template download widgets ---
upload_widget = widgets.FileUpload(accept='.xlsx', multiple=False)
download_button = widgets.Button(description="📄 Download Excel Template", button_style="info")
continue_button = widgets.Button(description="➡️ Continue to Simulation", button_style="success", disabled=True)
output = widgets.Output()

# --- Create Excel Template ---
def create_template_excel():
    data = {
        "Lane": ["Example-001"],
        "Order": ["ORD-0001"],
        "RequestedDeliveryDate": ["2025-10-21"],
        "Origin": ["US4819"],
        "OriginName": ["Houston Plant"],
        "ShipTo": ["US9021"],
        "ShipToName": ["Dallas Customer"],
        "TransitTime(minutes)": [180],
        "Miles": [240],
        "LoadingTimeOrigin": [60],
        "UnloadingTimeDestination": [45],
    }
    df_template = pd.DataFrame(data)
    df_template.to_excel("Orders_Template.xlsx", index=False)
    return "Orders_Template.xlsx"

def download_template(b):
    with output:
        clear_output()
        path = create_template_excel()
        print("📥 Template generated successfully:")
        display(FileLink(path))

download_button.on_click(download_template)

# --- Handle File Upload ---
def handle_upload(change):
    with output:
        clear_output()
        uploaded = list(upload_widget.value.values()) if isinstance(upload_widget.value, dict) else upload_widget.value
        if not uploaded:
            print("⚠️ No file uploaded yet.")
            return

        fileinfo = uploaded[0]
        filename = fileinfo.get("metadata", {}).get("name", fileinfo.get("name", "uploaded_file.xlsx"))
        print(f"✅ File '{filename}' uploaded successfully.")

        global df
        df = pd.read_excel(io.BytesIO(fileinfo["content"]))
        print(f"📊 Loaded {len(df)} rows and {len(df.columns)} columns.")
        display(df.head())

        continue_button.disabled = False

upload_widget.observe(handle_upload, names="value")

# --- Simulation Function ---
def run_simulation(data):
    with output:
        clear_output()
        print("🚀 Running truck allocation simulation...\n")

        data["RoundTripMinutes"] = (
            data["TransitTime(minutes)"] * 2
            + data["LoadingTimeOrigin"]
            + data["UnloadingTimeDestination"]
        )

        grouped = data.groupby("Origin").agg(
            Orders=("Order", "count"),
            AvgDistance=("Miles", "mean"),
            AvgRoundTripTime=("RoundTripMinutes", "mean")
        ).reset_index()

        grouped["TrucksNeeded"] = (grouped["Orders"] / 3).apply(lambda x: max(1, round(x)))
        grouped["AverageTPU"] = grouped["Orders"] / grouped["TrucksNeeded"]
        grouped["AverageWorkedHours"] = grouped["AvgRoundTripTime"] / 60
        grouped["TruckUtilization(%)"] = (grouped["AverageTPU"] / 3 * 100).round(1)

        print("✅ KPI Summary by Origin:")
        display(grouped)

        # --- Visualization ---
        plt.figure(figsize=(8,4))
        plt.bar(grouped["Origin"].astype(str), grouped["TruckUtilization(%)"], color="skyblue")
        plt.title("Average Truck Utilization by Origin")
        plt.xlabel("Origin")
        plt.ylabel("Utilization (%)")b

# --- Connect Continue Button ---
def run_next_step(b):
    run_simulation(df)

continue_button.on_click(run_next_step)

# --- Display UI ---
display(widgets.VBox([
    widgets.HTML("<h3>📦 Step 1: Upload or Download Your Orders Excel File</h3>"),
    widgets.HBox([upload_widget, download_button]),
    output,
    continue_button
]))


In [2]:
with open("requirements.txt", "w") as f:
    f.write("voila\nipywidgets\npandas\nmatplotlib\nopenpyxl\n")
print("✅ requirements.txt created successfully.")


✅ requirements.txt created successfully.


# 🚚 Truck Utilization & Visibility Simulator

An interactive Voila dashboard that calculates truck requirements, utilization, and KPIs
based on your Excel order data.

### How to Run

Click below to launch the app (no installation needed):

👉 [Launch in Binder](https://mybinder.org/v2/gh/rbsanchezpd/cemex/HEAD?urlpath=voila/render/TruckSimulator.ipynb)


In [5]:
git add TruckSimulator.ipynb requirements.txt README.md
git commit -m "Add Voila truck utilization dashboard"
git push origin main

SyntaxError: invalid syntax (2670742281.py, line 1)